In [40]:
# import sys
# !conda install --yes --prefix {sys.prefix} pingouin

In [65]:
import datetime
import numpy as np
import math
import os
import pandas as pd
import pingouin as pg
import re
import seaborn as sns
import json
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    

os.chdir(r'C:\Users\User\GitHub\WebET_Analysis')
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [66]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Read Data

In [67]:
data_et = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et.csv')
data_trial = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial.csv')
data_subject = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_subject.csv')

print(pd.DataFrame([[len(data_et)], [len(data_trial)], [len(data_subject)]], 
                   columns=['length'],
                   index=['data_et', 'data_trial', 'data_subject'])
     )

              length
data_et       618566
data_trial     30362
data_subject      40


# data_trial_choice

In [ ]:
data_trial_choice = data_trial.loc[
    data_trial['trial_type']=='eyetracking-choice', 
    [
        'run_id', 'chinFirst', 
        'task_nr', 
        'trial_index', 'trial_type', 'withinTaskIndex', 
        'choiceTask_amountLeftFirst', 
        'option_topLeft', 'option_bottomLeft',
        'option_topRight', 'option_bottomRight',
        'key_press', 'trial_duration_exact',
        'window_width', 'window_height'
    ]
]

# Reformat choice options

In [10]:
def reformatChoiceOptions(data):
    data['amountLeft']=0
    data.loc[(data['choiceTask_amountLeftFirst']==1) &
             (data['withinTaskIndex'] <41), 
             'amountLeft'] = 1

    data['aSS'] = \
        data.loc[:, ["option_topRight", "option_bottomRight"]].values.min(1)
    data.loc[data['amountLeft'] == 1, 'aSS'] = \
        data.loc[data['amountLeft'] == 1, ["option_topLeft", "option_bottomLeft"]].values.min(1)
    
    data['aLL'] = \
        data.loc[:, ["option_topLeft", "option_bottomLeft"]].values.max(1)
    data.loc[data['amountLeft'] == 0, 'aLL'] = \
        data.loc[data['amountLeft'] == 0, ["option_topRight", "option_bottomRight"]].values.max(1)

    data.loc[:, "tSS"] = 0 
    
    data['tLL'] = \
        data.loc[:, ["option_topRight", "option_bottomRight"]].values.max(1)
    data.loc[data['amountLeft'] == 0, 'tLL'] = \
        data.loc[data['amountLeft'] == 0, ["option_topLeft", "option_bottomLeft"]].values.max(1)
    
    data['LL_top'] = (data["option_topLeft"] > data["option_bottomLeft"]).astype(int)
        
    data["choseTop"] = 0
    data.loc[(data["key_press"]==38), "choseTop"] = 1

    data["choseLL"] = 0
    data.loc[(data["choseTop"]==1) & (data["LL_top"] == 1), "choseLL"] = 1
    
    return(data)

data_trial_choice = reformatChoiceOptions(data_trial_choice)

# k

In [11]:
def k(aLL, aSS, tLL):
    k = ((aLL / aSS) - 1) / tLL
    return k

data_trial_choice['k'] = k(data_trial_choice['aLL'], data_trial_choice['aSS'], data_trial_choice['tLL']) 
data_trial_choice

,run_id,chinFirst,task_nr,trial_index,trial_type,withinTaskIndex,choiceTask_amountLeftFirst,option_topLeft,option_bottomLeft,option_topRight,...,window_height,amountLeft,aSS,aLL,tSS,tLL,LL_top,choseTop,choseLL,k
144,1,1.0,2.0,145.0,eyetracking-choice,1.0,0.0,30.0,0.0,4.0,...,1080.0,0,3.0,4.0,0,30.0,1,1,1,0.011111
147,1,1.0,2.0,148.0,eyetracking-choice,2.0,0.0,30.0,0.0,5.0,...,1080.0,0,3.5,5.0,0,30.0,1,1,1,0.014286
150,1,1.0,2.0,151.0,eyetracking-choice,3.0,0.0,0.0,180.0,2.5,...,1080.0,0,2.5,5.0,0,180.0,0,0,0,0.005556
153,1,1.0,2.0,154.0,eyetracking-choice,4.0,0.0,180.0,0.0,4.5,...,1080.0,0,4.0,4.5,0,180.0,1,0,0,0.000694
156,1,1.0,2.0,157.0,eyetracking-choice,5.0,0.0,15.0,0.0,4.5,...,1080.0,0,4.0,4.5,0,15.0,1,0,0,0.008333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30339,2009,0.0,3.0,495.0,eyetracking-choice,76.0,0.0,4.0,2.5,90.0,...,1080.0,0,0.0,90.0,0,4.0,1,1,1,inf
30342,2009,0.0,3.0,498.0,eyetracking-choice,77.0,0.0,4.5,1.0,180.0,...,1080.0,0,0.0,180.0,0,4.5,1,1,1,inf
30345,2009,0.0,3.0,501.0,eyetracking-choice,78.0,0.0,4.0,3.0,90.0,...,1080.0,0,0.0,90.0,0,4.0,1,1,1,inf
30348,2009,0.0,3.0,504.0,eyetracking-choice,79.0,0.0,2.0,4.5,0.0,...,1080.0,0,0.0,7.0,0,4.5,0,0,0,inf


# data_et_choice

In [14]:
temp = data_et.merge(data_trial_choice.loc[:, 
                                           [
                                               'run_id', 'trial_index', 'trial_type',
                                               'amountLeft', 'LL_top', 'withinTaskIndex'
                                           ]
                                          ], 
                    on=['run_id', 'trial_index'],
                    how='left')

data_et_choice = temp.loc[temp['trial_type']=='eyetracking-choice', 
                          [
                              'run_id', 'trial_index', 
                              'x', 'y', 't', 't_task',
                              'amountLeft', 'LL_top', 'withinTaskIndex'
                          ]
                         ] \
    .reset_index(drop=True)

# Look direction

In [15]:
def lookDirections(data):
    data["look_left"] = (data["x"] < 0.5).astype(int)
    data["look_top"] = (data["y"] < 0.5).astype(int)
    return data

data_et_choice = lookDirections(data_et_choice)

# AOIs

In [16]:
def addAOI(data): 
    aoiCenters = pd.DataFrame(
        [
            [(0.05+0.9*0.2), 0.25],
            [(0.05+0.9*0.8), 0.25],
            [(0.05+0.9*0.2), 0.75],
            [(0.05+0.9*0.8), 0.75]
        ], 
        columns = ['x', 'y'],
        index = ['TL', 'TR', 'BL', 'BR']
    )
    
    data['aoi'] = 0 
    for aoi in aoiCenters.index:
        data.loc[
            (
                (data['x'] > (aoiCenters.loc[aoi, 'x'] - 0.175)) & \
                (data['x'] < (aoiCenters.loc[aoi, 'x'] + 0.175)) & \
                (data['y'] > (aoiCenters.loc[aoi, 'y'] - 0.175)) & \
                (data['y'] < (aoiCenters.loc[aoi, 'y'] + 0.175))
             ), 'aoi'] = aoi
    return data 

data_et_choice = addAOI(data_et_choice)
data_et_choice['aoi'].unique()

array([0, 'TR', 'TL', 'BR', 'BL'], dtype=object)

In [17]:
def createAOIColumns(data):
    data['aoi_aLL'] = 0
    data['aoi_tLL'] = 0 
    data['aoi_aSS'] = 0 
    data['aoi_tSS'] = 0
    
    # If amounts are on the left side
    # If the gaze point is in the top option
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='TL')), 
             'aoi_aLL'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='TR')), 
             'aoi_tLL'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='BL')), 
             'aoi_aSS'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='BR')), 
             'aoi_tSS'] = 1
    
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='TL')), 
             'aoi_aSS'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='TR')), 
             'aoi_tSS'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='BL')), 
             'aoi_aLL'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='BR')), 
             'aoi_tLL'] = 1
    
    # If amounts are on the right side
    # If the gaze point is in the top option
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='TL')), 
             'aoi_tLL'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='TR')), 
             'aoi_aLL'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='BL')), 
             'aoi_tSS'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='BR')),
             'aoi_aSS'] = 1

    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='TL')), 
             'aoi_tSS'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='TR')), 
             'aoi_aSS'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='BL')), 
             'aoi_tLL'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='BR')), 
             'aoi_aLL'] = 1
    return data

data_et_choice = createAOIColumns(data_et_choice)

# Transitions between AOIs

In [18]:
def add_transition_type(data):
    data = data.loc[(pd.notna(data['aoi'])), :]
    data['newAOIIndex'] = 0
    data.loc[(data['aoi_aLL']==1), 'newAOIIndex'] = 1
    data.loc[(data['aoi_tLL']==1), 'newAOIIndex'] = 2
    data.loc[(data['aoi_aSS']==1), 'newAOIIndex'] = 4
    data.loc[(data['aoi_tSS']==1), 'newAOIIndex'] = 8
    data.sort_values(by=['run_id', 'withinTaskIndex'])
    # Add a 0 due to the way np.diff works
    data['transition_type'] = np.append(np.diff(data['newAOIIndex']), [0])
    data['transition_type'] = abs(data['transition_type']) 
    return(data)

def cleanTransitions(data):
    indices = []
    for subject in data['run_id'].unique():
        df_subj = data.loc[data['run_id']==subject, :]
        for trial in df_subj['withinTaskIndex'].unique():
            df_thisTrial = data.loc[(
                                    (data['run_id'] == subject) &
                                    (data['withinTaskIndex'] == trial)
                                ), 'transition_type']
            indices.append(df_thisTrial.index.max())
    # last gaze point of each trial
    data.loc[indices, 'transition_type'] = 0 
    return(data)

data_et_choice = add_transition_type(data_et_choice)

data_et_choice = cleanTransitions(data_et_choice)
# View(data_et_choice.tail(20))

data_et_choice

,run_id,trial_index,x,y,t,t_task,amountLeft,LL_top,withinTaskIndex,look_left,look_top,aoi,aoi_aLL,aoi_tLL,aoi_aSS,aoi_tSS,newAOIIndex,transition_type
0,1.0,145.0,0.627605,0.429164,509546.610,0.000,0.0,1.0,1.0,0,1,0,0,0,0,0,0,0
1,1.0,145.0,0.604833,0.425830,510013.380,466.770,0.0,1.0,1.0,0,1,0,0,0,0,0,0,1
2,1.0,145.0,0.622740,0.387974,510486.345,939.735,0.0,1.0,1.0,0,1,TR,1,0,0,0,1,1
3,1.0,145.0,0.592737,0.415015,510958.065,1411.455,0.0,1.0,1.0,0,1,0,0,0,0,0,0,0
4,1.0,145.0,0.498632,0.300594,511430.640,1884.030,0.0,1.0,1.0,1,1,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155999,2009.0,507.0,0.288021,0.317593,1438291.000,1212.000,0.0,1.0,80.0,1,1,TL,0,1,0,0,2,0
156000,2009.0,507.0,0.288021,0.317593,1438365.000,1286.000,0.0,1.0,80.0,1,1,TL,0,1,0,0,2,0
156001,2009.0,507.0,0.288021,0.317593,1438437.000,1358.000,0.0,1.0,80.0,1,1,TL,0,1,0,0,2,0
156002,2009.0,507.0,0.288021,0.317593,1438512.000,1433.000,0.0,1.0,80.0,1,1,TL,0,1,0,0,2,0


# Eye-Tracking indices

In [ ]:
def addOptionIndex(data):
    if "optionIndex" in data.columns:
        print("Option Index already added!")
        data_output = data
    else:
        grouped = data_et_choice.groupby(['run_id', 'trial_index']) \
            ['aoi', 'aoi_aSS', 'aoi_aLL', 'aoi_tSS', 'aoi_tLL'].count() \
            .reset_index() \
            .rename(columns={"aoi": "count"})
        grouped['gazePoints_immediate'] = (grouped['aoi_aSS'] + grouped['aoi_tSS'])
        grouped['gazePoints_delay'] = (grouped['aoi_aLL'] + grouped['aoi_tLL'])
        grouped['optionIndex'] = (grouped['gazePoints_immediate'] - grouped['gazePoints_delay']) / \
                                 (grouped['gazePoints_immediate'] + grouped['gazePoints_delay'])
        data_output = data.merge(grouped[['run_id', 'trial_index', 'optionIndex']], 
                                          on=['run_id', 'trial_index'])
    return(data_output)

data_trial_choice = addOptionIndex(data_trial_choice)

In [ ]:
def addAttributeIndex(data):
    if "attributeIndex" in data.columns:
        print('Attribute Index already added!')
        data_output = data
    else:
        grouped = data_et_choice.groupby(['run_id', 'trial_index']) \
            ['aoi', 'aoi_aSS', 'aoi_aLL', 'aoi_tSS', 'aoi_tLL'].count() \
            .reset_index() \
            .rename(columns={"aoi": "count"})
        grouped['gazePoints_amount'] = (grouped['aoi_aLL'] + grouped['aoi_aSS'])
        grouped['gazePoints_time'] = (grouped['aoi_tLL'] + grouped['aoi_tSS'])
        grouped['attributeIndex'] = (grouped['gazePoints_amount'] - grouped['gazePoints_time']) / \
                                 (grouped['gazePoints_amount'] + grouped['gazePoints_time'])

        data_output = data.merge(grouped[['run_id', 'trial_index', 'attributeIndex']], 
                                          on=['run_id', 'trial_index'])
    return(data_output)

data_trial_choice = addAttributeIndex(data_trial_choice)

transition_count = pd.pivot_table(data_et_choice.loc[:, ['run_id', 'trial_index', 'transition_type']], 
                    index = ['run_id', 'trial_index'],
                    columns = ['transition_type'], 
                    aggfunc = len,
                    fill_value = 0
                    ).reset_index()

In [ ]:
def addTransition_type(data_trial, data_et):
    data_et.loc[:, 'transition_type'] = data_et.loc[:, 'transition_type'] \
        .replace({8: 0})
    if len(data_et.loc[:, 'transition_type'].unique())<2:
        transition_count = pd.DataFrame(
            columns = [
                'run_id', 'trial_index', 
                "trans_type_0",
                "trans_type_aLLtLL",
                "trans_type_tLLaSS",
                "trans_type_aLLaSS",
                "trans_type_aSStSS",
                "trans_type_tLLtSS",
                "trans_type_aLLtSS"
            ]
        )
    else:
        
        transition_count = pd.pivot_table(
            data_et.loc[:, ['run_id', 'trial_index', 'transition_type']], 
            index = ['run_id', 'trial_index'],
            columns = ['transition_type'], 
            aggfunc = len,
            fill_value = 0
        ) \
            .reset_index() \
            .rename(columns={
            0: "trans_type_0",
            1: "trans_type_aLLtLL",
            2: "trans_type_tLLaSS",
            3: "trans_type_aLLaSS",
            4: "trans_type_aSStSS",
            6: "trans_type_tLLtSS",
            7: "trans_type_aLLtSS"
        })

    if "trans_type_0" in data_trial:
        print("Transitions already added!")
        data_trial = data_trial
    else:
        data_trial = data_trial.merge(transition_count, on=['run_id', 'trial_index']) 
    return(data_trial)

data_trial_choice = addTransition_type(data_trial_choice, data_et_choice)

In [ ]:
def addPayneIndex(data):
    if "payneIndex" in data.columns:
        print("PayneIndex already added!")
    else: 
        # Option-wise: amount_LL-delay_LL = 1; amount_SS - delay_SS = 4
        # Attribute-wise: amount_LL-amount_SS = 3; delay_LL - delay_SS = 6
        # Cross: amount_LL-delay_SS = 7; delay_LL - amount_SS = 2
        optionWise_transition = data.loc[:, 'trans_type_aLLtLL'] + data.loc[:, 'trans_type_aSStSS']
        attributeWise_transition = data.loc[:, 'trans_type_aLLaSS'] + data.loc[:, 'trans_type_tLLtSS']  
        data['payneIndex'] = (optionWise_transition - attributeWise_transition) / \
            (optionWise_transition + attributeWise_transition)      
    return(data)

data_trial_choice = addPayneIndex(data_trial_choice)
data_trial_choice.columns

# Export data

In [ ]:
if not os.path.exists('./data_jupyter'):
    os.mkdir('./data_jupyter')

data_et_choice.to_csv("data_jupyter/data_et_choice.csv", index=False, header=True)
data_trial.to_csv("data_jupyter/data_trial.csv", index=False, header=True)
data_trial_choice.to_csv("data_jupyter/data_trial_choice.csv", index=False, header=True)
data_subject.to_csv("data_jupyter/data_subject.csv", index=False, header=True)

MatLab input

In [ ]:
if not os.path.exists('./amasino_dataPrep/data_source'):
    os.mkdir('./amasino_dataPrep/data_source')

data_et_choice['fixationCounter'] = 1
data_et_choice.loc[:, 
                       [
                           'run_id', 
                           'withinTaskIndex', 
                           'x', 
                           'y', 
                           't_task'
                       ]
                  ] \
   .to_csv("amasino_dataPrep/data_source/schneegansEtAl_ET.csv", index=False, header=False)

In [ ]:
data_trial_choice.loc[:, 
                       [
                           'run_id', 
                           'withinTaskIndex', 
                           'optionIndex', 
                           'attributeIndex', 
                           'payneIndex'
                       ]
                  ] \
    .fillna(0) \
    .to_csv("amasino_dataPrep/intermediateCSVs/ET_indices.csv", index=False, header=False)

In [ ]:
data_trial_choice.loc[:, 
                          [
                              'run_id', 
                              'aSS', 
                              'aLL', 
                              'tSS', 
                              'tLL', 
                              'choseLL', 
                              'trial_duration_exact', 
                              'LL_top',
                              'choseTop'
                          ]
                     ] \
    .to_csv("amasino_dataPrep/data_source/schneegansEtAl_behavior.csv", index=False, header=False)

In [ ]:
data_subject.loc[:, ['run_id', 'choseLL']] \
    .to_csv("amasino_dataPrep/intermediateCSVs/percLeft.csv", index=False, header=False)

# Feedback

In [ ]:
print('Success! Script ran through')